In [3]:
!uv add duckdb pandas pyarrow

Resolved 120 packages in 2.45s                                       
Prepared 1 package in 601ms                                              
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 36ms                                 
 + duckdb==1.4.4


In [66]:
import duckdb

con = duckdb.connect()

green = con.execute("""
    SELECT * 
    FROM read_parquet('green_tripdata_2025-11.parquet') 
""").df()

zones = con.execute("""
    SELECT * 
    FROM read_csv_auto('taxi_zone_lookup.csv')
""").df()

df = con.execute("""
    SELECT
        pu.Zone  AS pickup_zone,
        dz.Zone  AS dropoff_zone,
        tip_amount
    FROM read_parquet('green_tripdata_2025-11.parquet') AS g

    LEFT JOIN read_csv_auto('taxi_zone_lookup.csv') AS pu
        ON g.PULocationID = pu.LocationID

    LEFT JOIN read_csv_auto('taxi_zone_lookup.csv') AS dz
        ON g.DOLocationID = dz.LocationID
    where pickup_zone = 'East Harlem North'
    order by tip_amount desc
""").df()


In [27]:
green

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1,74,42,1,0.74,7.20,...,0.5,1.94,0.0,NaN,1.0,11.64,1,1,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1,74,42,2,0.95,7.20,...,0.5,0.00,0.0,NaN,1.0,9.70,2,1,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1,83,160,1,2.19,13.50,...,0.5,5.00,0.0,NaN,1.0,21.00,1,1,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1,166,127,1,5.44,24.70,...,0.5,0.50,0.0,NaN,1.0,27.70,1,1,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1,166,262,1,3.20,18.40,...,1.5,1.00,0.0,NaN,1.0,24.65,1,1,2.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,NaN,<NA>,59,51,<NA>,8.50,33.22,...,0.5,0.00,0.0,NaN,1.0,34.72,<NA>,<NA>,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,NaN,<NA>,74,151,<NA>,1.73,13.86,...,0.5,0.77,0.0,NaN,1.0,16.13,<NA>,<NA>,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,NaN,<NA>,33,163,<NA>,7.52,38.42,...,0.5,1.00,0.0,NaN,1.0,44.42,<NA>,<NA>,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,NaN,<NA>,16,95,<NA>,5.61,24.67,...,0.5,0.00,0.0,NaN,1.0,26.17,<NA>,<NA>,NaN,0.00


In [28]:
zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,N/A


In [67]:
df

,pickup_zone,dropoff_zone,tip_amount
0,East Harlem North,Yorkville West,81.89
1,East Harlem North,LaGuardia Airport,50.00
2,East Harlem North,East Harlem North,45.00
3,East Harlem North,Long Island City/Queens Plaza,34.25
4,East Harlem North,NA,28.90
...,...,...,...
12044,East Harlem North,East Harlem South,0.00
12045,East Harlem North,Long Island City/Queens Plaza,0.00
12046,East Harlem North,East Harlem North,0.00
12047,East Harlem North,Lenox Hill West,0.00
